In [ ]:
# vit_infer_basic.py
import torch
from torchvision import models
from torchvision.models import ViT_B_16_Weights

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 1) Tải ViT-B/16 pretrained
weights = ViT_B_16_Weights.DEFAULT
vit = models.vit_b_16(weights=weights).to(DEVICE)
vit.eval()

# 2) Transform chuẩn theo trọng số (resize 224, normalize…)
preprocess = weights.transforms()

# 3) Lấy một ảnh mẫu (dùng từ weights cho nhanh) và dự đoán
from PIL import Image
img = Image.open(weights.meta["categories_img"]) if "categories_img" in weights.meta else Image.open(requests.get("https://ultralytics.com/images/bus.jpg", stream=True).raw)  # fallback nếu thiếu

x = preprocess(img).unsqueeze(0).to(DEVICE)  # [1,3,224,224]
with torch.no_grad():
    logits = vit(x)                           # [1,1000]
probs = logits.softmax(dim=-1)[0]

# 4) In Top-5
idx2name = weights.meta["categories"]
top5 = probs.topk(5)
for p, i in zip(top5.values.tolist(), top5.indices.tolist()):
    print(f"{idx2name[i]}: {p:.3f}")
